# Guide to tHMM

In [1]:
import numpy as np
import scipy.stats as sp

In [2]:
obs = sp.gamma.rvs(a=0.5, scale=10.0, size=300)
a,b,c = sp.gamma.fit(obs)
print(a,b,c)
import matplotlib.pyplot as plt
plt.hist(obs)



0.5371951851738067 5.992087643027054e-08 8.548154393589002


(array([201.,  46.,  21.,  10.,   7.,   9.,   1.,   2.,   2.,   1.]),
 array([5.99208764e-08, 4.31504468e+00, 8.63008930e+00, 1.29451339e+01,
        1.72601785e+01, 2.15752232e+01, 2.58902678e+01, 3.02053124e+01,
        3.45203570e+01, 3.88354016e+01, 4.31504462e+01]),
 <a list of 10 Patch objects>)

### Synthesizing Cells (not required by the user)

In [3]:
from lineage.CellVar import CellVar as c
from lineage.CellVar import _double

In [4]:
T = np.array([[1.0, 0.0],
              [0.0, 1.0]])
    
parent_state = 1
parent_cell = c(state=parent_state, left=None, right=None, parent=None, gen=1)
left_cell, right_cell = parent_cell._divide(T)

In [5]:
print(left_cell, parent_cell.left)


 Generation: 2, State: 1, Observation: This cell has no observations to report. 
 Generation: 2, State: 1, Observation: This cell has no observations to report.


In [6]:
print(right_cell, parent_cell.right)


 Generation: 2, State: 1, Observation: This cell has no observations to report. 
 Generation: 2, State: 1, Observation: This cell has no observations to report.


## Creating a synthetic lineage (required by the user) "Two State Model"

In [7]:
from lineage.LineageTree import LineageTree
from lineage.StateDistribution import StateDistribution, get_experiment_time

### Creating a lineage and setting the full lineage (unpruned) as the one to be used

The required probabilities are those that define the tree and act of state switching. This process works by first creating a hidden tree of empty cells. Empty cells are those that have their states set but do not have any observations attached to them. We then draw as many observations from each state distribution and assign those observations to those cells. The $\pi$ and $T$ parameters are easy to define. The number of states is $k$. We require for $\pi$ a $k\times 1$ list of probabilities. These probabilities must add up to $1$ and they should be either in a $1$-dimensional list or a $1$-dimensional numpy array. The $T$ parameter should be a square numpy matrix of size $k\times k$. The rows are the states in which we are transitioning from and the columns are the states in which we are transitioning to. Each row of $T$ should sum to $1$. The columns need not sum to $1$.

In [8]:
# pi: the initial probability vector
pi = np.array([0.6, 0.4], dtype="float")

# T: transition probability matrix
T = np.array([[0.85, 0.15],
              [0.15, 0.85]])

The emission matrix $E$ is a little more complicated to define because this is where the user has complete freedom in defining what type of observation they care about. In particular, the user has to first begin with defining what observation he or she will want in their cells in their synthetic images. For example, if one is observing kinematics or physics, they might want to use Gaaussian distribution observations. In defining the random variables, the user will pull from a Gaussian distribution based on the mean and standard deviation of the different states he or she picks. They can also utilize the Gaussian probability distribution to define the likelihood as well. Furthermore, they can build an analytical estimator for their state distributions that yield the parameter estimates when given a list of observations. Finally, the user can also define a prune rule, which is essentially a boolean function that inspects a cell's observations and returns True if the cell's subtree (all the cells that are related to the cell in question and are of older generation) is to be pruned or False if the cell is safe from pruning. In the Gaussian example, a user can remove a cell's subtree if its observation is higher or lower than some fixed value.

We have already built, as an example, and as bioengineers, a model that resembles lineage trees. In our synthetic model, our emissions are multivariate. This first emission is a Bernoulli observation, $0$ implying death and $1$ implying division. The second and third emissions are continuous and are from exponential and gamma distributions respectively. Though these can be thought of cell lifetime's or periods in a certain cell phase, we want the user to know that these values can really mean anything and they are completely free in choosing what the emissions and their values mean. We define ways to calculate random variables for these multivariate observations and likelihoods of an observations. We also provide as a prune rule, keeping with the cell analogy, that if a cell has a $0$ in its Bernoulli observation, then its subtree is pruned from the full lineage tree. Though this will obviously introduce bias into estimation, we keep both the full tree and the pruned tree in the lineage objects, in the case a user would like to see the effects of analyzing on one versus the other.

Ultimately, $E$ is defined as a $k\times 1$ size list of state distribution objects. These distribution objects are rich in what they can already do, and a user can easily add more to their functionality. They only need to be instantiated by what parameters define that state's distribution.

In [9]:
# E: states are defined as StateDistribution objects

# State 0 parameters "Resistant"
state0 = 0
bern_p0 = 0.99
gamma_a1 = 5.0
loc = 0
gamma_scale1 = 1.0

# State 1 parameters "Susceptible"
state1 = 1
bern_p1 = 0.88
gamma_a2 = 10.0
gamma_scale2 = 2.0

state_obj0 = StateDistribution(state0, bern_p0, gamma_a1, loc, gamma_scale1)
state_obj1 = StateDistribution(state1, bern_p1, gamma_a2, loc, gamma_scale2)

E = [state_obj0, state_obj1]

The final required parameters are more obvious. The first is the desired number of cells one would like in their full unpruned lineage tree. This can be any number. The lineage tree is built 'from left to right'. What this means is that, we construct the binary tree by going to the left-most cell, dividing then walking through the generation. For example, if someone requested for

In [10]:
desired_num_cells = 2**10 - 1 
prune_boolean = False # To get the full tree

In [11]:
lineage1 = LineageTree(pi, T, E, desired_num_cells, prune_boolean)
print(lineage1)

This tree is NOT pruned. It is made of 2 states.
 For each state in this tree: 
 	 There are 520 cells of state 0, 
 	 There are 503 cells of state 1.
 This UNpruned tree has 1023 cells in total


### Obtaining how long the experiment ran by checking the time length of the longest branch

In [12]:
longest_branch_time = get_experiment_time(lineage1)
print(longest_branch_time)

224.99464338777142


### Estimation of distribution parameters using our estimators for full lineage

In [13]:
for state in range(lineage1.num_states):
    print("State {}:".format(state))
    print("                    estimated state", E[state].estimator(lineage1.lineage_stats[state].full_lin_cells_obs))
    print("original parameters given for state", E[state])

State 0:
5.432614614911175 0.0 0.9362804668514008
                    estimated state State object w/ parameters: 0.982692307692122, 5.432614614911175, 0.0, 0.9362804668514008.
original parameters given for state State object w/ parameters: 0.99, 5.0, 0, 1.0.
State 1:
9.650593362895044 0.0 2.081275375408776
                    estimated state State object w/ parameters: 0.8767395626241047, 9.650593362895044, 0.0, 2.081275375408776.
original parameters given for state State object w/ parameters: 0.88, 10.0, 0, 2.0.


### Estimation of distribution parameters using our estimators for pruned lineage

In [14]:
for state in range(lineage1.num_states):
    print("State {}:".format(state))
    print("                    estimated state", E[state].estimator(lineage1.lineage_stats[state].pruned_lin_cells_obs))
    print("original parameters given for state", E[state])

State 0:
5.224907390745023 0.0 0.9690772241004604
                    estimated state State object w/ parameters: 0.9831325301202491, 5.224907390745023, 0.0, 0.9690772241004604.
original parameters given for state State object w/ parameters: 0.99, 5.0, 0, 1.0.
State 1:
9.403700370342806 0.0 2.1357092764376455
                    estimated state State object w/ parameters: 0.8867403314914991, 9.403700370342806, 0.0, 2.1357092764376455.
original parameters given for state State object w/ parameters: 0.88, 10.0, 0, 2.0.


### Analyzing our first full lineage

In [15]:
from lineage.Analyze import Analyze, accuracyG
import copy as cp
X = [lineage1] # population just contains one lineage
states = [cell.state for cell in lineage1.output_lineage]
print(states)
deltas, state_ptrs, all_states, tHMMobj, NF, LL = Analyze(X, 2) # find two states


[0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 

In [16]:
from lineage.Analyze import accuracy
accuracy(tHMMobj, all_states)

[98.33822091886609]

### Estimated Markov parameters ($\pi$, $T$, $E$)

In [17]:
print(tHMMobj.estimate.pi)

[1.00000000e+00 3.99563184e-22]


In [18]:
print(tHMMobj.estimate.T)

[[0.84398591 0.15601409]
 [0.16476124 0.83523876]]


In [19]:
for state in range(tHMMobj.numStates):
    print(tHMMobj.estimate.E[state])

State object w/ parameters: 0.9845857418109886, 5.746152993988994, 0.0, 0.8738255117072706.
State object w/ parameters: 0.8749999999998512, 10.255233144722968, 0.0, 1.9622217974513658.


## Trying another lineage, this time pruning branches with ancestors that die

In [20]:
desired_num_cells = 2**12 -1 
prune_boolean = True # To get pruned tree

In [21]:
lineage2 = LineageTree(pi, T, E, desired_num_cells, prune_boolean)
print(lineage2)

This tree is pruned. It is made of 2 states.
 For each state in this tree: 
 	 There are 1185 cells of state 0, 
 	 There are 886 cells of state 1.
 This pruned tree has 2071 cells in total


In [22]:
longest2 = get_experiment_time(lineage2)
print(longest2)

261.9017430199332


### Estimation of distribution parameters using our estimators for pruned lineage

In [23]:
for state in range(lineage2.num_states):
    print("State {}:".format(state))
    print("                    estimated state", E[state].estimator(lineage2.lineage_stats[state].pruned_lin_cells_obs))
    print("original parameters given for state", E[state])

State 0:
4.995394098919204 0.0 1.019757665886551
                    estimated state State object w/ parameters: 0.9848101265821966, 4.995394098919204, 0.0, 1.019757665886551.
original parameters given for state State object w/ parameters: 0.99, 5.0, 0, 1.0.
State 1:
10.58028978496594 0.0 1.9403964179555453
                    estimated state State object w/ parameters: 0.8544018058689946, 10.58028978496594, 0.0, 1.9403964179555453.
original parameters given for state State object w/ parameters: 0.88, 10.0, 0, 2.0.


### Analyzing a population of lineages

In [26]:
X = [lineage1, lineage2] # population just contains one lineage
deltas, state_ptrs, all_states, tHMMobj, NF, LL = Analyze(X, 2) # find two states

3.261037577621692 0.0 0.3283787050585743
1.7802230102105059 0.0 6.722665168588085
1.768498087866998 0.0 6.757278073665475
1.768498087866998 0.0 6.757278073665475
1.768498087866998 0.0 6.757278073665475
1.768498087866998 0.0 6.757278073665475


### Estimated Markov parameters ($\pi$, $T$, $E$)

In [27]:
print(tHMMobj.estimate.pi)

[3.18833908e-05 9.99968117e-01]


In [28]:
print(tHMMobj.estimate.T)

[[1.86674612e-10 1.00000000e+00]
 [5.40603665e-05 9.99945940e-01]]


In [29]:
for state in range(tHMMobj.numStates):
    print(tHMMobj.estimate.E[state])

State object w/ parameters: 0.5, 10, 0, 1.
State object w/ parameters: 0.9295410471880783, 1.768498087866998, 0.0, 6.757278073665475.


## Creating a synthetic lineage that has three states

Here we generate a lineage with three states, which would be 1) Susciptible 2) Middle State 3) Resistant. The aim here is to show the transition from susciptible to resistant state doesn't happen immediately, and there is a gradual transition which is modeled as a middle state. The point to be considered here is that transition from 1 to 3 or otherwise is not possible so the probability of these transitions are zero, and most likely the initial cells are in susciptible state.

**State 1**: Susceptible

**State 2**: Transition state

**State 3**: Resistant state


In [30]:
# pi: the initial probability vector
pi_3 = np.array([0.5, 0.25, 0.25])

# T: transition probability matrix
T_3 = np.array([[0.65, 0.35, 0.00],
                [0.20, 0.40, 0.40],
                [0.00, 0.10, 0.90]])

In [33]:
# E: states are defined as StateDistribution objects

# State 0 parameters "Susciptible"
state0 = 0
bern_p0 = 0.7
expon_scale_beta0 = 20
gamma_a0 = 5.0
loc = 0.0
gamma_scale0 = 1.0

# State 1 parameters "Middle state"
state1 = 1
bern_p1 = 0.85
expon_scale_beta1 = 60
gamma_a1 = 10.0
gamma_scale1 = 2.0

# State 2 parameters "Resistant"
state2 = 2
bern_p2 = 0.99
expon_scale_beta2 = 80
gamma_a2 = 15.0
gamma_scale2 = 3.0

state_obj0 = StateDistribution(state0, bern_p0, gamma_a0, loc, gamma_scale0)
state_obj1 = StateDistribution(state1, bern_p1, gamma_a1, loc, gamma_scale1)
state_obj2 = StateDistribution(state2, bern_p2, gamma_a2, loc, gamma_scale2)

E_3 = [state_obj0, state_obj1, state_obj2]

In [34]:
desired_num_cells = 2**13 - 1 
prune_boolean = False # To get the full tree

In [35]:
lineage3 = LineageTree(pi_3, T_3, E_3, desired_num_cells, prune_boolean)
print(lineage3)

This tree is NOT pruned. It is made of 3 states.
 For each state in this tree: 
 	 There are 1231 cells of state 0, 
 	 There are 1641 cells of state 1, 
 	 There are 5319 cells of state 2.
 This UNpruned tree has 8191 cells in total


In [36]:
longest3 = get_experiment_time(lineage3)
print(longest3)

609.7772093987023


### Estimation of distribution parameters using our estimators for full lineage (3 state)

In [38]:
for state in range(lineage3.num_states):
    print("State {}:".format(state))
    print("estimated state", E_3[state].estimator(lineage3.lineage_stats[state].full_lin_cells_obs))
    print("estimated state", E_3[state].estimator(lineage3.lineage_stats[state].pruned_lin_cells_obs))
    print("true_____ state", E_3[state])

State 0:
5.04529871761122 0.0 0.9803901076542363
estimated state State object w/ parameters: 0.7181153533712075, 5.04529871761122, 0.0, 0.9803901076542363.
nan 0.0 nan
estimated state State object w/ parameters: 9.999999998e-11, nan, 0.0, nan.
true_____ state State object w/ parameters: 0.7, 5.0, 0.0, 1.0.
State 1:
10.004163710025631 0.0 1.971677797889958
estimated state State object w/ parameters: 0.8586227909810653, 10.004163710025631, 0.0, 1.971677797889958.
estimated state State object w/ parameters: 0.5, 10, 0, 1.
true_____ state State object w/ parameters: 0.85, 10.0, 0.0, 2.0.
State 2:
15.207497350058313 0.0 2.954715524090452
estimated state State object w/ parameters: 0.990223726264317, 15.207497350058313, 0.0, 2.954715524090452.
estimated state State object w/ parameters: 0.5, 10, 0, 1.
true_____ state State object w/ parameters: 0.99, 15.0, 0.0, 3.0.


### Analyzing a three state lineage

In [ ]:
X = [lineage3] # population just contains one lineage

deltas, state_ptrs, all_states, tHMMobj, NF, LL = Analyze(X, 3) # find three states

2.12244281637112 0.0 16.005408827245574
8.17528169820048 0.0 0.22254948441964692
2.1752378351248227 0.0 15.723202816023937
2.5231189869524946 0.0 13.898497096076586
12.704617715849217 0.0 0.18350529340014654
2.377692946866412 0.0 14.432549244761345
3.021888940189655 0.0 11.969320211362124
13.516991347904526 0.0 0.2023509534095782
2.306432702778159 0.0 14.695458066109262
3.9934250656670094 0.0 9.40994485249951
10.954061266328749 0.0 0.30142403738764273
2.436295155097663 0.0 13.978700683507125
5.623401909083976 0.0 6.972942310460473
7.708314553803018 0.0 0.5290742491853416
3.150316735288953 0.0 11.194990352064014
7.1115797226581945 0.0 5.685190375171449
5.665106802429848 0.0 0.8305700321200887
4.2090023864562855 0.0 8.609580423841821
8.13313793798497 0.0 5.088557694147963
4.800486748670226 0.0 1.0587702970061559
4.627621708167341 0.0 7.822350975346733
8.942370287925625 0.0 4.702088170748879
4.127045261016889 0.0 1.3089571392096533
4.820019395999445 0.0 7.499457938884841
9.711782158055984

In [ ]:
accuracy(tHMMobj, all_states)

### Estimated Markov parameters ($\pi$, $T$, $E$)

In [ ]:
print(tHMMobj.estimate.pi)

In [ ]:
print(tHMMobj.estimate.T)

In [ ]:
for state in range(tHMMobj.numStates):
    print(tHMMobj.estimate.E[state].shape[0])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
A = sp.gamma.rvs(a=20,scale=4,size=2)
B = sp.gamma.rvs(a=5,scale=1,size=1000)
plt.hist(A)
plt.figure()
plt.hist(B)

In [ ]:
import scipy.stats as sp
gamma_ll = sp.gamma.pdf(x=A, a=a, scale=b)  # gamma likelihood
print(gamma_ll)